In [1]:
import pandas as pd
import numpy as np
import math
from statistics import mean
import pickle
from scipy import sparse
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import re
import geopy.distance

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>


In [2]:
# double check the path on harddrive
import pathlib
print(pathlib.Path().resolve())

/Users/kennethshinn/Documents/fyoudie/python_workbooks


In [3]:
# this is critical and sensitive BING API stuff!!!

import csv
import requests
import json
import math

api_key = "AgUSM867a3r-7GFqCUQ81nvWLngFtrbanaBzA41qVEDoN-PFSEQiCTtx0eY9aJ--"

In [4]:
# location functions

def get_address(name, lattitude, longitude):
    lat_long_string = str(lattitude) + ',' + str(longitude)
    response = requests.get("https://dev.virtualearth.net/REST/v1/LocalSearch/",
                            params={"query": name,
                                    "userLocation": lat_long_string,
                                    "key": api_key})
    data = response.json()
    location_data = data['resourceSets'][0]['resources'][0]["Address"]
    return location_data


def get_current_location(lattitude, longitude):
    lat_long_string = str(lattitude) + ',' + str(longitude)
    response = requests.get("https://dev.virtualearth.net/REST/v1/Locations/" + lat_long_string + "?",
                            params={"key": api_key})
    data = response.json()
    location_data = data['resourceSets'][0]['resources'][0]["address"]
    return location_data


def get_distance_time_name(name, lattitude, longitude, trans_method):
    lat_long_string = str(lattitude) + ',' + str(longitude)
    dest_address = get_address(name, lattitude, longitude)["formattedAddress"]

    if (trans_method == "Driving") | (trans_method == "Transit"):
        optimization = "time"
    else:
        optimization = "distance"

    response = requests.get("https://dev.virtualearth.net/REST/v1/Routes/" + str(trans_method) + "?",
                            params={"o": "json",
                                    "wp.0": lat_long_string,
                                    "wp.1": dest_address,
                                    "distanceUnit": "Mile",
                                    "optimize": optimization,
                                    "key": api_key})
    data = response.json()
    direction_data = data['resourceSets'][0]["resources"][0]
    return direction_data

def get_distance_time_address(address, lattitude, longitude, trans_method):
    lat_long_string = str(lattitude) + ',' + str(longitude)

    if (trans_method == "Driving") | (trans_method == "Transit"):
        optimization = "time"
    else:
        optimization = "distance"

    response = requests.get("https://dev.virtualearth.net/REST/v1/Routes/" + str(trans_method) + "?",
                            params={"o": "json",
                                    "wp.0": lat_long_string,
                                    "wp.1": address,
                                    "distanceUnit": "Mile",
                                    "optimize": optimization,
                                    "key": api_key})
    data = response.json()
    direction_data = data['resourceSets'][0]["resources"][0]
    return direction_data

# returns nice values for distance (in miles) and time (in minutes of travel)
def get_distance_time_name_values(name, lattitude, longitude, trans_method):
    results = get_distance_time_name(name, lattitude, longitude, trans_method)
    distance = math.ceil(results["travelDistance"] * 10) / 10
    time = math.ceil(results["travelDuration"] / 60)
    return distance, time

# returns nice values for distance (in miles) and time (in minutes of travel)
def get_distance_time_address_values(address, lattitude, longitude, trans_method):
    results = get_distance_time_address(address, lattitude, longitude, trans_method)
    distance = math.ceil(results["travelDistance"] * 10) / 10
    time = math.ceil(results["travelDuration"] / 60)
    return distance, time



In [5]:
import json

# import restaurant data from files

business_file_path = "../yelp_dataset/yelp_academic_dataset_business.json"

business_info_dict = {}
with open(business_file_path, 'rb') as business_data:
    for b in business_data:
        b_dict = json.loads(b)

        bid = b_dict['business_id']
        category = b_dict["categories"]
        if (category is not None and "Restaurants" in category):
            business_info_dict[bid] = b_dict

business_info_df = pd.DataFrame(business_info_dict.values())

In [6]:
rid_dict = pd.read_pickle("../script_data/new_to_old_restaurant_ids.pkl")
rid_dict = {v:k for k,v in rid_dict.items()}

In [7]:
with open("../pickles/very_big_masks.pkl","rb") as f:
    mask_dict = pickle.load(f)

# # need to process this to be able to use it
# mask_vectors = list(mask_dict.values())
mask_vectors = [i.todense() for i in mask_dict]
mask_matrix = np.stack(mask_vectors)

/var/folders/kk/pr3cm19s0pnf27_x6wbg037m0000gn/T/ipykernel_18062/4280266897.py:2: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  mask_dict = pickle.load(f)


In [8]:
# this will pull the feature vectors for the restaurants

with open('../pickles/rid_feature_dict.pkl',"rb") as f:
    rid_feature_dict = pickle.load(f)
    
# this will pull the feature reweights (since due to the masks, some of the weights over power)
with open('../pickles/feature_weight_corrections.pkl',"rb") as f:
    feature_reweights = pickle.load(f)

In [9]:
# we have to post process the vectors so that we extract a parameter term that is weighted or unweighted
# price = weighted
# taste = weighted 
# noise = unweighted
# decor = weighted
# saltiness = unweighted
# msg = unweighted 
# cleaniless = weighted
# oiliness = unweighted

weighted = np.matrix([1,1,0,1,0,0,1,0])
unweighted = np.matrix([1,1,1,1,1,1,1,1]) - weighted

rid_feature_dict_combined = {}
for key, value in rid_feature_dict.items():
    # here, we apply the feature reweights
    rid_feature_dict_combined[key] = np.multiply(feature_reweights.T, np.multiply(unweighted, value[0].T) +  np.multiply(weighted, value[1].T))


In [10]:
# matching these with the people vectors, test
from numpy import genfromtxt
user_features = np.matrix(genfromtxt("../user_restaurant_data/user_features_test.csv", delimiter=','))
# normalize the user features if they aren't already
user_features = user_features / np.linalg.norm(user_features)


# Order is price, taste, noise, decor, saltiness, msg, cleanliness, and oiliness
# for every person in this file, dot the person with the feature vectors, and sort
user_sorted_rid = {}
cnt = 0
for row in user_features:
    print(row)
    user_feature_vector = row
    rid_scores = []
    for key, value in rid_feature_dict_combined.items():
        rid_scores.append((key, float(np.dot(user_feature_vector,value.T))))
    user_sorted_rid[cnt] = sorted(rid_scores, key=lambda x: x[1], reverse = True)
    cnt += 1

[[ 0.         0.4472136 -0.4472136  0.4472136  0.        -0.4472136
   0.4472136  0.       ]]


In [11]:
# lattitude, longitude
user_locations = np.matrix(genfromtxt("../user_restaurant_data/user_locations_test.csv", delimiter=','))


In [12]:
person_no = 0

In [13]:
# convert the user sorted rids into a pandas data frame
rids = [x for (x,y) in user_sorted_rid[person_no]]
scores = [y for (x,y) in user_sorted_rid[person_no]]
    
data = {'rid': rids, 'score': scores}
rids_scores = pd.DataFrame.from_dict(data)

In [14]:
rids = []
business_ids = []

for key, value in rid_dict.items():
    rids.append(key)
    business_ids.append(value)
    
data = {'rid': rids, 'business_id': business_ids}
rids_ogrids = pd.DataFrame.from_dict(data)

In [15]:
# this is the master recommendations

df_master_recs_all = pd.merge(business_info_df, pd.merge(rids_scores, rids_ogrids, on='rid', how='inner'), on='business_id', how = 'inner')



In [16]:
restaurant_locations = list(zip(list(df_master_recs_all['latitude']),list(df_master_recs_all['longitude'])))

distances = []
for lat, long in restaurant_locations:
    coords_1 = (lat, long)
    coords_2 = (np.array(user_locations)[0][0], np.array(user_locations)[0][1])
    distances.append(geopy.distance.distance(coords_1, coords_2).mi)

df_master_recs_all['distance'] = distances

In [17]:
# get all restaurants within the distance threshold
distance_threshold = 20 # miles
df_master_recs = df_master_recs_all.loc[df_master_recs_all['distance'] < distance_threshold]

In [18]:
df_master_recs = df_master_recs.sort_values(by=['score'],ascending=False).head(30)

In [19]:
# add the driving distance to all possible recs
rec_names = list(df_master_recs['name'])

travel_distance = []
travel_times  = []

lat = np.array(user_locations)[0][0]
lng = np.array(user_locations)[0][1]
travel_method = 'Driving'

for i in range(len(df_master_recs)):
    try: # try to query the address
        address = df_master_recs.iloc[i]['address']
        city = df_master_recs.iloc[i]['city']
        state = df_master_recs.iloc[i]['state']
        address_string = address + ', ' + city + ', ' + state
        distance, time = get_distance_time_address_values(address_string, lat, lng, travel_method)
        travel_distance.append(distance)
        travel_times.append(time)
    except:
        try: 
            name = df_master_recs.iloc[i]['name']
            # first try querying just the name
            distance, time = get_distance_time_name_values(name, lat, lng, travel_method)
            travel_distance.append(distance)
            travel_times.append(time)
        except: # failure, just add None
            travel_distance.append(None)
            travel_times.append(None)
            

In [20]:
df_master_recs['travel_distance'] = travel_distance
df_master_recs['travel_time'] = travel_times

In [21]:
df_master_recs['score_rank'] = list(df_master_recs['score'].rank(ascending=False))
df_master_recs['travel_time_rank'] = list(df_master_recs['travel_time'].rank(ascending=True))
df_master_recs['combined_rank'] = df_master_recs['score_rank'] + df_master_recs['travel_time_rank']
df_master_recs = df_master_recs.sort_values(by=['combined_rank'],ascending=True)

In [22]:
df_master_recs

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,categories,hours,rid,score,distance,travel_distance,travel_time,score_rank,travel_time_rank,combined_rank
26810,odk7Qmifs1B2b_fVf4z1qQ,Thai Thani,11025 International Dr,Orlando,FL,32821,28.405760,-81.455344,4.0,811,...,"Thai, Noodles, Soup, Restaurants","{'Monday': '11:30-23:0', 'Tuesday': '11:30-23:...",42387,0.032325,5.720396,7.9,18,1.0,7.0,8.0
39453,wpm_IUt_AoZOXdUvPdS1Ew,Hungry Pants,3421 S Orange Ave,Orlando,FL,32806,28.505815,-81.375879,5.0,214,...,"Comfort Food, Wraps, Vegetarian, Sandwiches, R...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-20:0', ...",6849,0.031498,8.368618,12.6,24,3.0,14.0,17.0
42379,AFjTPvLlpcoPWsTtK7Im2A,Kathi Rolls,4205 Curry Ford Rd,Orlando,FL,32806,28.524692,-81.332293,4.5,99,...,"Food, Halal, Wraps, Pizza, Indian, Juice Bars ...","{'Tuesday': '12:30-19:30', 'Wednesday': '12:30...",21838,0.031743,11.280752,16.5,25,2.0,15.5,17.5
35492,ewGeqOJShmU9B3Wph0ckKQ,Taco Maker Mexican Grill,"9924 Universal Blvd, Ste 204",Orlando,FL,32819,28.427343,-81.444636,3.5,42,...,"Restaurants, Mexican, Food, Latin American, Br...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-20:0', ...",1178,0.029262,5.058130,6.8,18,11.0,7.0,18.0
7712,ZZV3d2MxJzTPFHFCKv7Uag,168 Chinese,7721 Turkey Lake Rd,Orlando,FL,32819,28.453639,-81.477504,3.5,10,...,"Chinese, Restaurants",None,4235,0.027715,2.365387,3.7,12,19.0,2.0,21.0
7737,DIHSgOkVRQQqjEKNZ9i5vA,DOMU - Orlando,"7600 Dr Phillips Blvd, Ste 14, Dr. Phillips",Orlando,FL,32819,28.452718,-81.489892,4.5,245,...,"Japanese, Ramen, Restaurants, Asian Fusion, Ni...","{'Monday': '0:0-0:0', 'Tuesday': '17:0-22:0', ...",46373,0.027466,1.867160,2.8,10,21.0,1.0,22.0
29708,9J479DEA3meQbkvcpSNogQ,JuicyPot,5206 W Colonial Dr,Orlando,FL,32808,28.551986,-81.450687,5.0,68,...,"Cajun/Creole, Seafood, Hot Pot, Restaurants, F...","{'Tuesday': '12:0-22:0', 'Wednesday': '12:0-22...",36595,0.029260,6.416498,10.1,20,12.0,10.5,22.5
16480,WH1ZFrHsxXGioOyspbl03w,NaraDeva Thai Restaurant,4696 Millenia Plz Way,Orlando,FL,32839,28.494384,-81.429018,4.0,348,...,"Ethnic Food, Food, Thai, Vegetarian, Specialty...","{'Monday': '11:0-21:30', 'Tuesday': '11:0-21:3...",19056,0.028459,5.043143,8.2,18,16.0,7.0,23.0
20091,GoPpSNXPojXZffKY3cQPXg,The Mexican Camel,5752 International Dr,Orlando,FL,32819,28.461396,-81.458011,4.5,341,...,"Mediterranean, Halal, Vegan, Mexican, Restaurants","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",33579,0.027626,3.206053,5.6,14,20.0,3.0,23.0
40747,oW0u2tZX0aslP_OoRalNtQ,Cabimas Latin Restaurant & Gallery,2132 Whisper Lakes Blvd,Orlando,FL,32837,28.396777,-81.405554,4.5,6,...,"American (New), Restaurants, Latin American, S...","{'Monday': '11:30-21:0', 'Tuesday': '11:30-21:...",26456,0.028573,8.234053,11.6,20,14.0,10.5,24.5


In [23]:
# create a filename based on the input
# input is location
# threshold distance
# driving vs. no driving
# user vector
filename_base = 'restaurant_recs_'
filename = filename_base + str(np.array(user_locations)[0][0]) + "_" + str(np.array(user_locations)[0][1]) + "_" + str(distance_threshold) + "_" + str(travel_method) + "__" + '_'.join(str(elem) for elem in user_features.tolist()[0]) + '.tsv' 

df_master_recs.to_csv('../sample_recs/' + filename, sep='\t')
